In [1]:
import numba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.special import gammaln
from scipy.optimize import minimize
from scipy.stats import circmean, circstd

import healpy as hp
from astropy.table import Table
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord as SkyCoord

In [2]:
def galactic_cut(data,RA_column_index,Dec_column_index,Lim):

    data_array = np.array(data)

    ra_column_array = data_array[:,RA_column_index]
    dec_column_array = data_array[:,Dec_column_index]

    c = SkyCoord(ra_column_array,dec_column_array, frame='icrs', unit='deg')

    b_array = np.array(c.galactic.b)

    Index = 0
    Index_list = []
    for b in b_array:
        if b > -Lim  and b < Lim:
            Index_list.append(Index)
        Index+=1

    index_to_delete = np.array(Index_list)
    new_data_array = np.delete(data_array,index_to_delete,axis=0)
    new_data = pd.DataFrame(new_data_array,columns=data.columns.values)

    return new_data

def mapa_simulado_mask(density_map):

    map = []
    for Np in density_map:

        N_simulado = np.random.poisson(Np)
        map.append(N_simulado)
    # Convertendo density_map para float64 para suportar hp.UNSEEN
    map = np.array(map)
    novo_map = map.astype(np.float64)
    # Criando a máscara para os pixels onde não há fontes (valor zero)
    mask = novo_map > 0
    # Aplicando a máscara ao mapa para criar um mapa mascarado
    masked_map = np.copy(novo_map)
    masked_map[~mask] = hp.UNSEEN
    return masked_map, np.sum(map)

def mapa_simulado_unmask(density_map):

    map = []
    for Np in density_map:

        N_simulado = np.random.poisson(Np)
        map.append(N_simulado)
    # Convertendo density_map para float64 para suportar hp.UNSEEN
    map = np.array(map)
    novo_map = map.astype(np.float64)
    return novo_map, np.sum(novo_map)

def exclui_pixels_vazios(density_map,thetas_pixels,phis_pixels):
    
    indices_para_excluir = np.where(density_map==0)[0]
    density_map = np.delete(density_map,indices_para_excluir)

    thetas_pixels = np.delete(thetas_pixels,indices_para_excluir)
    phis_pixels = np.delete(phis_pixels,indices_para_excluir)

    n_x = np.sin(thetas_pixels)*np.cos(phis_pixels)
    n_y = np.sin(thetas_pixels)*np.sin(phis_pixels)
    n_z = np.cos(thetas_pixels)

    n_vec = np.stack([n_x,n_y,n_z],axis=1)

    return density_map, n_vec

def estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num):
    print('Estimador χ2')

    l_lista=[]
    b_lista=[]
    relativo =[]
    absoluto = []
    for _ in range(num):

        simulado, total_fontes = mapa_simulado_unmask(density_map)
        Ni1, n_vec1 = exclui_pixels_vazios(simulado,thetas_pixels,phis_pixels)

        #print('Quantidade de fontes:',total_fontes)
        #print('Novo array density_map:',Ni1)
        #print('Quantidade de pixels visiveis:',len(Ni1))

        def chi2(params):
            M0 = params[0]
            d_vec = params[1:]
            
            N_i_model = N_i_model = M0 + np.dot(n_vec1,d_vec)
            
            chi2_value = np.sum((Ni1 - N_i_model)**2 / Ni1)
            
            return chi2_value

        M1, d_vec1 = np.mean(simulado), np.array([0.01, 0.01, 0.01])

        #print('') 
        #print('Minimizando χ2')
        initial_guess = [M1, d_vec1[0], d_vec1[1], d_vec1[2]]

        #print('Rodando uma única vez (Método Nelder-Mead)')
        result1 = minimize(chi2, initial_guess, method='Nelder-Mead')

        M0_best1, Dx_best1, Dy_best1, Dz_best1 = result1.x[0], result1.x[1], result1.x[2], result1.x[3]

        # anisotropia_dipolar:
        D_modulo_relativo1 = np.sqrt((Dx_best1**2)+(Dy_best1**2)+(Dz_best1**2))/M0_best1
        #print(f'|D_relativo|={D_modulo_relativo1:.4f}')

        D_modulo_absoluto1 = np.sqrt(Dx_best1**2 + Dy_best1**2 + Dz_best1**2)
        #print(f'|D_absoluto|={D_modulo_absoluto1:.3f}')

        n1 = np.array([Dx_best1,Dy_best1,Dz_best1])/np.linalg.norm(D_modulo_absoluto1)

        theta_n1, phi_n1 = hp.vec2ang(n1)
        l_result1, b_result1  = np.degrees(phi_n1), 90 - np.degrees(theta_n1)

        #print(l_result1, b_result1)
        l_lista.append(l_result1)
        b_lista.append(b_result1)
        relativo.append(D_modulo_relativo1)
        absoluto.append(D_modulo_absoluto1)


    D_abs_mean = np.mean(np.array(absoluto))
    D_rel_mean = np.mean(np.array(relativo))

    D_abs_std = np.std(np.array(absoluto))
    D_rel_std = np.std(np.array(relativo))
    
    print(f'|D_relativo|={D_rel_mean:.5f}±{D_rel_std:4f}')
    print(f'|D_absoluto|={D_abs_mean:.4f}±{D_abs_std:.4f}')
    
    # Latitude Galáctica
    l_lista = np.array(l_lista)
    l_rad = np.radians(l_lista)
    # Latitude Média
    l_mean_rad = circmean(l_rad, high=2*np.pi, low=0)
    l_mean_deg = np.degrees(l_mean_rad)
    # Latitude desvio 
    l_std_rad = circstd(l_rad, high=np.pi, low=-np.pi)
    l_std_deg = np.degrees(l_std_rad)
    #print(f'l: {l_mean_deg:.2f}°')
    #print(f'dl: {l_std_deg:.2f}°')

    # Longitude Galáctica
    b_lista = np.array(b_lista)
    b_rad = np.radians(b_lista)
    # Longitude Média
    b_mean_rad = circmean(b_rad, high=np.pi, low=-np.pi)
    b_mean_deg = np.degrees(b_mean_rad)
    # Longitude desvio
    b_std_rad = circstd(b_rad, high=np.pi, low=-np.pi)
    b_std_deg = np.degrees(b_std_rad)
    #print(f'b: {b_mean_deg:.2f}°')
    #print(f'db:{b_std_deg:.2f}°')

    print(f'(l,b) = {l_mean_deg:.2f}°±{l_std_deg:.2f}°, {b_mean_deg:.2f}°±{b_std_deg:.2f}°')
    return 

def estimador_likelihood(density_map,thetas_pixels,phis_pixels,num):
    print('Estimador Log likelihood')
    
    l_lista=[]
    b_lista=[]
    relativo =[]
    absoluto = []
    for _ in range(num):

        simulado, total_fontes = mapa_simulado_unmask(density_map)
        Ni1, n_vec1 = exclui_pixels_vazios(simulado,thetas_pixels,phis_pixels)
        M1, d_vec1 = np.mean(simulado), np.array([0.01, 0.01, 0.01])

        def negativo_log_likelihood(params):
    
            M = params[0]
            d_vec = params[1:]
            
            lambdas = M*(1+np.dot(n_vec1,d_vec))

            if np.any(lambdas <= 0):
                return 1e10

            logP = Ni1*np.log(lambdas)-lambdas - gammaln(Ni1+1)
            
            likelihood_value = -np.sum(logP)
            return likelihood_value
        
        #print('') 
        #print('Minimizando χ2')
        initial_guess = [M1, d_vec1[0], d_vec1[1], d_vec1[2]]

        #print('Rodando uma única vez (Método Nelder-Mead)')
        result1 = minimize(negativo_log_likelihood, initial_guess, method='Nelder-Mead')
        M0_best1, Dx_best1, Dy_best1, Dz_best1 = result1.x[0], result1.x[1], result1.x[2], result1.x[3]

        # anisotropia_dipolar:
        D_modulo_relativo1 = np.sqrt((Dx_best1**2)+(Dy_best1**2)+(Dz_best1**2))/M0_best1
        #print(f'|D_relativo|={D_modulo_relativo1:.4f}')

        D_modulo_absoluto1 = np.sqrt(Dx_best1**2 + Dy_best1**2 + Dz_best1**2)
        #print(f'|D_absoluto|={D_modulo_absoluto1:.3f}')

        n1 = np.array([Dx_best1,Dy_best1,Dz_best1])/np.linalg.norm(D_modulo_absoluto1)

        theta_n1, phi_n1 = hp.vec2ang(n1)
        l_result1, b_result1  = np.degrees(phi_n1), 90 - np.degrees(theta_n1)

        #print(l_result1, b_result1)
        l_lista.append(l_result1)
        b_lista.append(b_result1)
        relativo.append(D_modulo_relativo1)
        absoluto.append(D_modulo_absoluto1)

    D_abs_mean = np.mean(np.array(absoluto))
    D_rel_mean = np.mean(np.array(relativo))

    D_abs_std = np.std(np.array(absoluto))
    D_rel_std = np.std(np.array(relativo))

    print(f'|D_relativo|={D_rel_mean:.5f}±{D_rel_std:4f}')
    print(f'|D_absoluto|={D_abs_mean:.4f}±{D_abs_std:.4f}')
    
    # Latitude Galáctica
    l_lista = np.array(l_lista)
    l_rad = np.radians(l_lista)
    # Latitude Média
    l_mean_rad = circmean(l_rad, high=2*np.pi, low=0)
    l_mean_deg = np.degrees(l_mean_rad)
    # Latitude desvio 
    l_std_rad = circstd(l_rad, high=np.pi, low=-np.pi)
    l_std_deg = np.degrees(l_std_rad)
    #print(f'l: {l_mean_deg:.2f}°')
    #print(f'dl: {l_std_deg:.2f}°')

    # Longitude Galáctica
    b_lista = np.array(b_lista)
    b_rad = np.radians(b_lista)
    # Longitude Média
    b_mean_rad = circmean(b_rad, high=np.pi, low=-np.pi)
    b_mean_deg = np.degrees(b_mean_rad)
    # Longitude desvio
    b_std_rad = circstd(b_rad, high=np.pi, low=-np.pi)
    b_std_deg = np.degrees(b_std_rad)
    #print(f'b: {b_mean_deg:.2f}°')
    #print(f'db:{b_std_deg:.2f}°')

    print(f'(l,b) = {l_mean_deg:.2f}°±{l_std_deg:.2f}°, {b_mean_deg:.2f}°±{b_std_deg:.2f}°')
    return 

In [3]:
def densidade_pixel(nside,catalog,ra='',dec=''):
    
    npix = hp.nside2npix(nside)
    # Obtendo os angulos dos pixels em coordenadas esféricas padrão em radianos
    thetas_pixels, phis_pixels = hp.pix2ang(nside, np.arange(npix))

    # obtendo as coordenadas cartesianas de cada fontes
    C_nvss = SkyCoord(catalog[ra],catalog[dec], frame='icrs', unit='deg')
    l, b  = C_nvss.galactic.l.value, C_nvss.galactic.b.value
    phis_fontes, thetas_fontes = np.radians(l), np.radians(90.0 - b)

    pix_indices = hp.ang2pix(nside, thetas_fontes, phis_fontes)
    # Contando o número de fontes em cada pixel
    density_map = np.bincount(pix_indices, minlength=npix)

    return density_map, thetas_pixels, phis_pixels

In [ ]:
name = 'NVSS'
filename = 'NVSS_Catalog.fits'
hdu = fits.open(filename)
nvss = hdu[1].data
nvss = Table(nvss)
nvss = nvss.to_pandas()

# Corte de Fluxo 20 mJy < S < 1000 mJy
nvss = nvss[(20 <= nvss['FLUX_20_CM']) & (nvss['FLUX_20_CM'] <= 1000)]

nside = 64

num = 1_000
#--------------------------------------------------------------------------
cut_b = 10
nvss = galactic_cut(nvss,0,1,cut_b)

print('|b|<10')

density_map, thetas_pixels, phis_pixels = densidade_pixel(nside, nvss,'RA','DEC')
estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num)
estimador_likelihood(density_map,thetas_pixels,phis_pixels,num)


#--------------------------------------------------------------------------
print('\n')

cut_b = 20
nvss = galactic_cut(nvss,0,1,cut_b)
print('|b|<20')

density_map, thetas_pixels, phis_pixels = densidade_pixel(nside, nvss,'RA','DEC')
estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num)
estimador_likelihood(density_map,thetas_pixels,phis_pixels,num)

#--------------------------------------------------------------------------
print('\n')

cut_b = 30
nvss = galactic_cut(nvss,0,1,cut_b)

print('|b|<30')

density_map, thetas_pixels, phis_pixels = densidade_pixel(nside, nvss,'RA','DEC')
estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num)
estimador_likelihood(density_map,thetas_pixels,phis_pixels,num)


|b|<10
Estimador χ2


In [ ]:
name = 'TGSS'
filename = 'TGSS_Catalog.fits'
hdu = fits.open(filename)
tgss = hdu[1].data
tgss = Table(tgss)
tgss = tgss.to_pandas()

# Corte de Fluxo 100 mJy < S < 5000 mJ
tgss = tgss[(100 <= tgss['Total_flux']) & (tgss['Total_flux'] <= 5000)]

num = 1_000
#--------------------------------------------------------------------------
cut_b = 10
tgss = galactic_cut(tgss,0,1,cut_b)

print('|b|<10')

density_map, thetas_pixels, phis_pixels = densidade_pixel(nside, tgss,'RA','DEC')
estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num)
estimador_likelihood(density_map,thetas_pixels,phis_pixels,num)


#--------------------------------------------------------------------------
cut_b = 20
tgss = galactic_cut(tgss,0,1,cut_b)
print('|b|<20')

density_map, thetas_pixels, phis_pixels = densidade_pixel(nside, tgss,'RA','DEC')
estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num)
estimador_likelihood(density_map,thetas_pixels,phis_pixels,num)

#--------------------------------------------------------------------------
print('\n')

cut_b = 30
tgss = galactic_cut(tgss,0,1,cut_b)

print('|b|<30')

density_map, thetas_pixels, phis_pixels = densidade_pixel(nside, tgss,'RA','DEC')
estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num)
estimador_likelihood(density_map,thetas_pixels,phis_pixels,num)

FileNotFoundError: [Errno 2] No such file or directory: 'TGSS_Catalog.fits'

In [18]:
filename = 'quaia_G20.5.fits'
hdu = fits.open(filename)
hi = hdu[1].data
hi = Table(hi)
hi = hi.to_pandas()

nside = 64

num = 1000
#--------------------------------------------------------------------------
b = hi['b']
cut_b = 10
mask = np.where((-cut_b > b) | (b > cut_b))[0]
data10 = hi.loc[mask]

print('|b|<10')

density_map, thetas_pixels, phis_pixels = densidade_pixel(nside, data10,'ra','dec')
estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num)
estimador_likelihood(density_map,thetas_pixels,phis_pixels,num)


#--------------------------------------------------------------------------
print('\n')

b = hi['b']
cut_b = 20
mask = np.where((-cut_b > b) | (b > cut_b))[0]
data20 = hi.loc[mask]

print('|b|<20')

density_map, thetas_pixels, phis_pixels = densidade_pixel(nside, data20,'ra','dec')
estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num)
estimador_likelihood(density_map,thetas_pixels,phis_pixels,num)

#--------------------------------------------------------------------------
print('\n')

b = hi['b']
cut_b = 30
mask = np.where((-cut_b > b) | (b > cut_b))[0]
data30 = hi.loc[mask]


print('|b|<30')

density_map, thetas_pixels, phis_pixels = densidade_pixel(nside, data30,'ra','dec')
estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num)
estimador_likelihood(density_map,thetas_pixels,phis_pixels,num)

|b|<10
Estimador χ2
|D_relativo|=0.12502±0.027204
|D_absoluto|=2.3950±0.5228
(l,b) = 250.80°±22.90°, 39.43°±20.26°
Estimador Log likelihood
|D_relativo|=0.00227±0.000055
|D_absoluto|=0.0723±0.0018
(l,b) = 207.63°±1.82°, 32.62°±1.24°


|b|<20
Estimador χ2
|D_relativo|=0.07547±0.007790
|D_absoluto|=2.0742±0.2150
(l,b) = 285.62°±28.21°, 57.83°±10.57°
Estimador Log likelihood
|D_relativo|=0.00150±0.000048
|D_absoluto|=0.0525±0.0017
(l,b) = 212.30°±2.92°, 41.68°±1.88°


|b|<30
Estimador χ2
|D_relativo|=0.06769±0.005602
|D_absoluto|=2.2374±0.1833
(l,b) = 311.60°±20.99°, 39.19°±12.55°
Estimador Log likelihood
|D_relativo|=0.00097±0.000044
|D_absoluto|=0.0357±0.0016
(l,b) = 250.20°±6.43°, 55.50°±3.23°


In [6]:
filename = 'quaia_G20.0.fits'
hdu = fits.open(filename)
low = hdu[1].data
low = Table(low)
low = low.to_pandas()

nside = 64

num = 1_000
#--------------------------------------------------------------------------
b = low['b']
cut_b = 10
mask = np.where((-cut_b > b) | (b > cut_b))[0]
data10 = low.loc[mask]

print('|b|<10')

density_map, thetas_pixels, phis_pixels = densidade_pixel(nside, data10,'ra','dec')
estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num)
estimador_likelihood(density_map,thetas_pixels,phis_pixels,num)


#--------------------------------------------------------------------------
print('\n')

b = low['b']
cut_b = 20
mask = np.where((-cut_b > b) | (b > cut_b))[0]
data20 = low.loc[mask]

print('|b|<20')

density_map, thetas_pixels, phis_pixels = densidade_pixel(nside, data20,'ra','dec')
estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num)
estimador_likelihood(density_map,thetas_pixels,phis_pixels,num)

#--------------------------------------------------------------------------
print('\n')

b = low['b']
cut_b = 30
mask = np.where((-cut_b > b) | (b > cut_b))[0]
data30 = low.loc[mask]


print('|b|<30')

density_map, thetas_pixels, phis_pixels = densidade_pixel(nside, data30,'ra','dec')
estimador_chi_erros(density_map,thetas_pixels, phis_pixels,num)
estimador_likelihood(density_map,thetas_pixels,phis_pixels,num)

|b|<10
Estimador χ2
|D_relativo|=0.13080±0.014413
|D_absoluto|=1.5839±0.1752
(l,b) = 250.99°±19.74°, 48.62°±8.62°
Estimador Log likelihood
|D_relativo|=0.00381±0.000117
|D_absoluto|=0.0713±0.0022
(l,b) = 212.50°±2.27°, 32.62°±1.58°


|b|<20
Estimador χ2
|D_relativo|=0.08200±0.007070
|D_absoluto|=1.2910±0.1112
(l,b) = 268.14°±31.28°, 62.34°±9.01°
Estimador Log likelihood
|D_relativo|=0.00257±0.000110
|D_absoluto|=0.0523±0.0022
(l,b) = 216.92°±3.77°, 41.14°±2.40°


|b|<30
Estimador χ2
|D_relativo|=0.07504±0.006881
|D_absoluto|=1.3423±0.1225
(l,b) = 327.50°±18.77°, 48.19°±9.17°
Estimador Log likelihood
|D_relativo|=0.00177±0.000112
|D_absoluto|=0.0379±0.0024
(l,b) = 252.04°±7.28°, 48.35°±3.91°


In [19]:
name = 'NVSS'
filename = 'NVSS_Catalog.fits'
hdu = fits.open(filename)
nvss = hdu[1].data
nvss = Table(nvss)
nvss = nvss.to_pandas()

# Corte de Fluxo 20 mJy < S < 1000 mJy
nvss = nvss[(20 <= nvss['FLUX_20_CM']) & (nvss['FLUX_20_CM'] <= 1000)]

cut_b = 20
nvss = galactic_cut(nvss,0,1,cut_b)

nside = 64
npix = hp.nside2npix(nside)
# Obtendo os angulos dos pixels em coordenadas esféricas padrão em radianos
thetas_pixels, phis_pixels = hp.pix2ang(nside, np.arange(npix))

# obtendo as coordenadas cartesianas de cada fontes
C_nvss = SkyCoord(nvss['RA'],nvss['DEC'], frame='icrs', unit='deg')
l, b  = C_nvss.galactic.l.value, C_nvss.galactic.b.value
phis_fontes, thetas_fontes = np.radians(l), np.radians(90.0 - b)

pix_indices = hp.ang2pix(nside, thetas_fontes, phis_fontes)
# Contando o número de fontes em cada pixel
density_map = np.bincount(pix_indices, minlength=npix)

In [13]:
estimador_chi_erros(density_map,10)

(l,b) = 232.19°±29.01°, 57.10°±14.82°


0

In [57]:
erros(density_map,10_000)

(l,b) = 224.67°±37.63°, 54.71°±13.77°


0

In [21]:
estimador_likelihood(density_map,thetas_pixels, phis_pixels,1_000)

Estimador Log likelihood
|D_relativo|=0.00278±0.000456
|D_absoluto|=0.0220±0.0036
(l,b) = 237.50°±15.58°, 35.66°±10.28°


In [31]:
estimador_chi_erros(density_map,10)

Estimador χ2
|D_relativo|=0.02969±0.007135
|D_absoluto|=0.1611±0.0385
(l,b) = 241.98°±35.05°, 49.40°±12.47°


0

In [33]:
estimador_likelihood(density_map,100)

Estimador Log likelihood
|D_relativo|=0.00282±0.000464
|D_absoluto|=0.0224±0.0037
(l,b) = 236.39°±17.66°, 35.42°±10.46°


0